# POMDPs

In [18]:
import stormpy
import stormpy.pomdp

program = stormpy.parse_prism_program('storm_files/maze2.prism')
prop = "R=? [F \"goal\"]"
properties = stormpy.parse_properties_for_prism_program(prop, program, None)
model = stormpy.build_model(program, properties)
print(model.model_type)

ModelType.POMDP


In [4]:
print(model.nr_observations)
for state in model.states:
    print("State {} has observation id {}".format(state.id, model.observations[state.id]))

8
State 0 has observation id 6
State 1 has observation id 1
State 2 has observation id 4
State 3 has observation id 7
State 4 has observation id 4
State 5 has observation id 3
State 6 has observation id 0
State 7 has observation id 0
State 8 has observation id 0
State 9 has observation id 0
State 10 has observation id 0
State 11 has observation id 0
State 12 has observation id 2
State 13 has observation id 5
State 14 has observation id 2


In [19]:
result = stormpy.model_checking(model, properties[0])
print("Result: {}".format(round(result.at(0), 6)))

ERROR (verification.h:341): The model type POMDP is not supported by the sparse engine.


RuntimeError: NotSupportedException: The model type POMDP is not supported by the sparse engine.

In [35]:
def build_pomdp(program):
    options = stormpy.BuilderOptions([])
    options.set_build_state_valuations()
    options.set_build_choice_labels()
    options.set_build_all_labels()
    options.set_build_all_reward_models()

    model =  stormpy.build_sparse_model_with_options(program, options)
    model = stormpy.pomdp.make_canonic(model)
    return model


prism_program = stormpy.parse_prism_program('storm_files/maze2.prism')
prop = stormpy.parse_properties_for_prism_program("R=? [F \"goal\"]", prism_program)[0]
model = build_pomdp(prism_program)
print(model.model_type)


ModelType.POMDP


In [41]:
options = stormpy.pomdp.IterativeQualitativeSearchOptions()
solver = stormpy.pomdp.create_iterative_qualitative_search_solver_Double(model,  prop.raw_formula, options)
result = solver.compute_winning_policy_for_initial_states(100)

RuntimeError: std::bad_cast

In [45]:
import subprocess

storm_dir = '/home/scarr/storm/'

subprocess.run(storm_dir+'build/bin/storm-pomdp --prism storm_files/maze2.prism --prop "Pmax=? [!\"bad\" U \"goal\"]" --qualitative-analysis -const "sl=0.3" --memlesssearch iterative')

FileNotFoundError: [Errno 2] No such file or directory: '/home/scarr/storm/build/bin/storm-pomdp --prism storm_files/maze2.prism --prop "Pmax=? [!"bad" U "goal"]" --qualitative-analysis -const "sl=0.3" --memlesssearch iterative'

# POMDP Shields

In [47]:
with open('storm_files/obstacle.nm') as f:
    print(f.read())

pomdp

observables
start
endobservables

const int N=6;
const int axMAX = N-1;
const int ayMAX = N-1;
const int axMIN = 0;
const int ayMIN = 0;
const double slippery = 0.1;
const int ob1x = axMAX-1;
const int ob1y = ayMAX-1;
const int ob2x = axMAX;
const int ob2y = ayMIN+1;
const int ob3x = axMIN+1;
const int ob3y = ayMIN;
const int ob4x = axMAX;
const int ob4y = ayMAX-1;
const int ob5x = axMAX-3;
const int ob5y = ayMAX-1;

formula done = start & ax = axMAX & ay = ayMAX;
observable "amdone" = done;
formula crash =  (ax = ob1x & ay = ob1y) | (ax = ob2x & ay = ob2y)  | (ax = ob3x & ay = ob3y) | (ax = ob4x & ay = ob4y) | (ax = ob5x & ay = ob5y)  ;
observable "hascrash" = crash;


module master
    start : bool init false;

    [placement] !start -> (start'=true);
    [north] start & !done -> true;
    [south] start  & !done -> true;
    [east] start  & !done-> true;
    [west] start & !done -> true;


endmodule


module robot
    ax : [axMIN..axMAX] init 0;
    ay : [ayMIN..ayMAX] init 0;

### Shield functions

In [59]:
def compute_winning_region(model, formula, initial=True):
    options = stormpy.pomdp.IterativeQualitativeSearchOptions()
    model = stormpy.pomdp.prepare_pomdp_for_qualitative_search_Double(model, formula)
    solver = stormpy.pomdp.create_iterative_qualitative_search_solver_Double(model, formula, options)
    if initial:
        solver.compute_winning_policy_for_initial_states(100)
    else:
        solver.compute_winning_region(100)
    return solver.last_winning_region

def construct_otf_shield(model, winning_region):
    return stormpy.pomdp.BeliefSupportWinningRegionQueryInterfaceDouble(model, winning_region)

def build_pomdp(program, formula):
    options = stormpy.BuilderOptions([formula])
    options.set_build_state_valuations()
    options.set_build_choice_labels()
    options.set_build_all_labels()
    options.set_build_all_reward_models()
    options.set_build_observation_valuations()
    return stormpy.build_sparse_model_with_options(program, options)


In [74]:
from model_simulator import Tracker
from model_simulator import SimulationExecutor

prism_program = stormpy.parse_prism_program('storm_files/obstacle.nm')
prop = stormpy.parse_properties_for_prism_program("Pmax=? [ \"notbad\" U \"goal\"]", prism_program)[0]
raw_formula = prop.raw_formula
model = build_pomdp(prism_program, raw_formula)
model = stormpy.pomdp.make_canonic(model)
winning_region = compute_winning_region(model, raw_formula, False)
otf_shield = construct_otf_shield(model, winning_region)
tracker = Tracker(model, otf_shield)



 WARN (IterativePolicySearch.cpp:376): This code is slow and should only be used for debugging.


In [75]:
from rlshield.recorder import LoggingRecorder, VideoRecorder, StatsRecorder
recorder = StatsRecorder(only_keep_finishers=False)
executor = SimulationExecutor(model, tracker)
executor.simulate(recorder, total_nr_runs=10, nr_good_runs=3, maxsteps=100)
recorder.save(output_path, f"{videoname}")


ValueError: too many values to unpack (expected 2)